In [1]:
# load dependencies'
import concurrent.futures
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import osmnx as ox
import networkx as nx
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from urllib.parse import urljoin
from shapely.geometry import Point, LineString, Polygon
import pyproj 
import mm_utils
from datetime import datetime
from IMM import IMM
from Junction_Condtion import JC
from MMJ import MMJ
# import sys
# sys.path.append('..')

In [2]:
# def point_matching(curr_loc, curr_edge):
#     # matched position to the current edge 
#     # input need to be panda series
#     # curr loc need attribute geometry point
#     # curr_edge need attribute 'geometry' lines
#     # output a point that matched to the current edge
#     crs_utm = curr_loc.crs
#     dist = curr_edge['geometry'].project(curr_loc['geometry']).iloc[0]
#     matched_point = list(curr_edge['geometry'].interpolate(dist).coords)
#     matched_point = gpd.GeoDataFrame(geometry=gpd.points_from_xy([matched_point[0][0]], [matched_point[0][1]]), crs= crs_utm)
#     return matched_point

In [3]:
def AHPMM(trajector_data, nodes_data, edges_data):
    matched_links = []
    # matched_points = []
    iter, previous_edge = IMM(trajectory_data, edges_data, iter=0)
    matched_links.append(previous_edge)
    while iter < len(trajectory_data):
        current_position = trajectory_data.iloc[[iter]]
        # print(current_position)
        if JC(trajectory_data, nodes_data, edges_data, current_position, previous_edge, iter):
            iter += 1
        else:
            tmp = MMJ(trajectory_data, nodes_data, edges_data, current_position, previous_edge, iter)
            iter, previous_edge = tmp
        matched_links.append(previous_edge)

        # matched_point = point_matching(current_position, previous_edge.iloc[0])
        # # find longitude and latitude for last matched data
        # matched_point['lon_lat'] = matched_point.to_crs({'init': 'epsg:4326'})
        # matched_points.append(matched_point)
    return matched_links #, matched_points

In [4]:
import sys
sys.path.append('..')
import preprocessing.downloader as dl
# id = dl.fetch_ids_on_envirocar(num_trajectories=3)[2]
id = "64c35f12c7e42807e7b8fc4e"
gdf = dl.download_from_envirocar(id, use_cache=False)
trajectory_data, nodes_data, edges_data = dl.to_fuzzy_AHP_input(gdf)

In [5]:
# trajectory_data = pd.read_pickle('gdf_utm.pkl')
# nodes_data = pd.read_pickle('nodes_utm.pkl')
# edges_data = pd.read_pickle('edges_utm.pkl')
# initial_previous_position = pd.read_pickle('gdf_utm.pkl').iloc[iter].to_frame()
# matched_links, matched_points = AHPMM(trajectory_data, nodes_data, edges_data)
matched_links = AHPMM(trajectory_data, nodes_data, edges_data)

In [6]:
matched_edge_df = pd.concat(matched_links)
# matched_points_df = pd.concat(matched_points)

In [9]:
# visualization
%matplotlib tk
# This is how we  visualize edges and error bound 

# plotting edges and starting point together 
f, ax = plt.subplots()

# location for all point
#locs_utm.plot(ax=ax)
point_locs = trajectory_data['geometry'].to_frame()
point_locs.plot(ax = ax, color="Red")


# this plot all the road system 
edges_data.plot(ax=ax)
# candidate_link.iloc[[0]].plot(ax = ax, color = 'Red')
# candidate_link.iloc[[3]].plot(ax = ax, color = 'Yellow')
# candidate_link.iloc[4:6].plot(ax = ax, color = 'Blue')
# candidate_link.iloc[6:8].plot(ax = ax)

# this plot the selected edge at time point 

# matched point plot
# matched_point_df.plot(ax = ax, color = "Green")
# next_point_matching.plot(ax = ax, color = "Green")
# nodes_data.plot(ax = ax, color = "Green")
# matched_edge
matched_edge_df.plot(ax = ax, color = "Yellow")
# matched_edge_df.iloc[[7]].plot(ax = ax, color = "Red")
# matched_edge_df.iloc[[26]].plot(ax = ax, color = "Yellow")
# # plot closest node
# closest_node.plot(ax = ax, color = "Black")
# closest_edge.plot(ax = ax , color = "Black")
# # debuging for djiksta shortest path
# origin_node.plot(ax = ax, color = "Black")
# target_node.plot(ax = ax , color = "Black")

# matched_points_df.plot(ax = ax, color = "Red")

<Axes: >